이번엔 이미 훈련되어져 있는 워드 임베딩을 불러서 임베딩 벡터로 사용하겠습니다. 훈련 데이터가 부족하다면 nn.Embedding()을 사용하는 것보다 나은 선택일 수 있습니다.

왜냐하면 훈련 데이터가 적다면 파이토치의 nn.Embedding()으로 해당 문제에 충분히 특화된 임베딩 벡터를 만들어내는 것이 쉽지 않기 때문입니다. 그렇기에 일반적이고 보다 많은 훈련 데이터로 훈련된 임베딩 벡터들을 사용합니다.

## **1. IMDB 리뷰 데이터를 훈련 데이터로 사용하기**

실습을 위해서 사전 훈련된 임베딩 벡터들을 맵핑시킬 대상인 훈련 데이터가 필요합니다. 여기서는 토치텍스트에서 제공하는 IMDB 리뷰 데이터를 다운받아 사용하겠습니다.

In [5]:
from torchtext.legacy import data, datasets

토치텍스트를 통한 실습을 진행하기 위해 우선 두 개의 Field 객체를 정의합니다.

In [6]:
TEXT = data.Field(sequential=True, batch_first=True, lower=True)
LABEL = data.Field(sequential=False, batch_first=True)

필드를 정의헀다면, 실습을 위해 필요한 데이터를 준비해야 합니다. torchtext.legacy.datasets은 IMDB, TREC(질문 분류), 언어 모델링(WikiText-2) 등 다른 여러 데이터셋을 제공합니다. 

In [7]:
trainset, testset = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


100%|██████████| 84.1M/84.1M [01:33<00:00, 901kB/s] 


현재 훈련 데이터에는 총 25,000개의 샘플이 존재합니다.

In [9]:
print("훈련 데이터의 크기: {}".format(len(trainset)))

훈련 데이터의 크기: 25000


첫 번째 샘플을 출력하면 다음과 같습니다.

In [10]:
print(vars(trainset[0]))

{'text': ['bromwell', 'high', 'is', 'a', 'cartoon', 'comedy.', 'it', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other', 'programs', 'about', 'school', 'life,', 'such', 'as', '"teachers".', 'my', '35', 'years', 'in', 'the', 'teaching', 'profession', 'lead', 'me', 'to', 'believe', 'that', 'bromwell', "high's", 'satire', 'is', 'much', 'closer', 'to', 'reality', 'than', 'is', '"teachers".', 'the', 'scramble', 'to', 'survive', 'financially,', 'the', 'insightful', 'students', 'who', 'can', 'see', 'right', 'through', 'their', 'pathetic', "teachers'", 'pomp,', 'the', 'pettiness', 'of', 'the', 'whole', 'situation,', 'all', 'remind', 'me', 'of', 'the', 'schools', 'i', 'knew', 'and', 'their', 'students.', 'when', 'i', 'saw', 'the', 'episode', 'in', 'which', 'a', 'student', 'repeatedly', 'tried', 'to', 'burn', 'down', 'the', 'school,', 'i', 'immediately', 'recalled', '.........', 'at', '..........', 'high.', 'a', 'classic', 'line:', 'inspector:', "i'm", 'here', 'to', 'sack', 'one', 'of', '

여기 있는 단어들을 각각 사전 훈련된 임베딩 벡터들로 맵핑해서 값을 부여해보겠습니다.

토치텍스트의 Field 객체의 build_vocab을 통해 사전 훈련된 워드 임베딩을 사용할 수 있습니다. 여기서는 직접 훈련시킨 사전 훈련된 워드 임베딩을 사용하는 방법과 토치텍스트에서 제공하는 사전 훈련된 워드 임베딩을 사용하는 두 가지 방법을 다룹니다.

## **2. 영어 Word2Vec 만들기**

파이썬의 gensim 패키지에는 Word2Vec을 지원하고 있어서 손쉽게 단어를 임베딩 벡터로 변환시킬 수 있습니다. 영어로 된 코퍼스를 다운받아 전처리를 수행하고, 전처리한 데이터를 바탕으로 Word2Vec 작업을 진행하겠습니다.

In [11]:
import re
import urllib.request
import zipfile
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize

### **1) 훈련 데이터 이해하기**

먼저 훈련 데이터를 다운받습니다.

In [12]:
# 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/09.%20Word%20Embedding/dataset/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x23a01c82f70>)

훈련 데이터 파일은 xml문법으로 작성되어 있습니다. 그렇기에 자연어를 얻기 위해 전처리가 필요합니다. 얻고자 하는 실질적 데이터는 영어문장으로만 구성된 내용을 담고 있는 <content>와 </content> 사이의 내용입니다. 전처리를 통해 xml문법들은 제거하고 해당 데이터만 가져와야 합니다. 또한 (Laughter)이나 (Applause)와 같은 배경음을 내는 단어도 제거해야 합니다.

### **2) 훈련 데이터 전처리하기**

In [13]:
targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
    tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
    normalized_text.append(tokens)

# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]

In [14]:
print("총 샘플의 개수 : {}".format(len(result)))

총 샘플의 개수 : 273380


약 27만 3천개의 샘플로 이루어져 있습니다. 또한 토큰화도 이미 이루어져있음을 확인할 수 있습니다.

In [15]:
# 샘플 3개만 출력
for line in result[:3]:
    print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


### **3) Word2Vec 훈련시키기**

In [19]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

model = Word2Vec(sentences=result, vector_size=100, window=5, min_count=5, workers=4, sg=0)

Woed2Vec의 하이퍼파라미터값은 다음과 같습니다.

- **size** = 워드 벡터의 특징 값, 즉 임베딩 된 벡터의 차원
- **window** = 컨텍스트 윈도우 크기
- **min_count** = 단어 최소 빈도 수 제한(빈도가 적은 단어들은 학습하지 않는다)
- **workers** = 학습을 위한 프로세스 수
- **sg** = 0은 CBOW, 1은 Skip-gram

Word2Vec에 대해서 학습을 진행했습니다. Word2Vec는 입력한 단어에 대해 가장 유사한 단어들을 출력하는 model.wv.most_similar를 지원합니다. man과 가장 유사한 단어들을 찾으면 다음과 같습니다.

In [20]:
model_result = model.wv.most_similar("man")
print(model_result)

[('woman', 0.8708652257919312), ('guy', 0.8115028738975525), ('lady', 0.7953709363937378), ('boy', 0.7691215872764587), ('girl', 0.7543208003044128), ('soldier', 0.7282536029815674), ('gentleman', 0.7186691761016846), ('kid', 0.7079837918281555), ('poet', 0.6812275648117065), ('photographer', 0.6759003400802612)]


유사한 단어와 유사한 정도를 확인할 수 있습니다.

### **4) Word2Vec 모델 저장하고 로드하기**

이제 학습한 모델을 저장하겠습니다. 

In [23]:
model.wv.save_word2vec_format('../model/eng_w2v') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format("../model/eng_w2v") # 모델 불러오기

로드한 모델에 대해서 다시 man과 유사한 단어를 출력하면 잘 작동하는 것을 확인할 수 있습니다.

In [24]:
model_result = loaded_model.most_similar("man")
print(model_result)

[('woman', 0.8708652257919312), ('guy', 0.8115028738975525), ('lady', 0.7953709363937378), ('boy', 0.7691215872764587), ('girl', 0.7543208003044128), ('soldier', 0.7282536029815674), ('gentleman', 0.7186691761016846), ('kid', 0.7079837918281555), ('poet', 0.6812275648117065), ('photographer', 0.6759003400802612)]


## **3. 토치텍스트를 사용한 사전 훈련된 워드 임베딩**

이번에는 토치텍스트를 사용해서 외부에서 가져온 사전 훈련된 워드 임베딩을 사용해보겠습니다.

### **1) 사전 훈련된 Word2Vec 모델 확인하기**

전 챕터에서 만든 모델을 이용하겠습니다. 

In [25]:
from gensim.models import KeyedVectors

word2vec_model = KeyedVectors.load_word2vec_format('../model/eng_w2v')

In [26]:
print(word2vec_model['this']) # 영어 단어 'this'의 임베딩 벡터값 출력

[-5.71524054e-02 -1.95141539e-01 -7.43177831e-01 -9.08190846e-01
 -3.99657071e-01  1.71754622e+00 -7.54990220e-01 -6.76760614e-01
 -2.45056093e-01 -3.89378488e-01  7.59430468e-01  1.80086350e+00
 -2.23722029e+00 -5.70789933e-01 -1.71727133e+00  1.11946940e+00
  2.59290725e-01  9.10938025e-01  7.70651221e-01  3.35467696e-01
  9.89516020e-01  1.45179653e+00  1.49299872e+00 -1.62867546e-01
  7.26294637e-01 -8.96329582e-01 -1.02155519e+00  3.92085940e-01
  1.31438684e+00  7.37661660e-01  1.31158435e+00 -3.60702425e-01
 -5.36255062e-01 -1.43435109e+00  2.75447786e-01 -1.53213596e+00
 -1.51507509e+00 -6.90128803e-02  3.19300830e-01 -1.31668293e+00
 -9.09678161e-01 -1.34827328e+00 -1.05471337e+00 -1.03209186e+00
  8.95805582e-02 -2.01313329e+00 -1.30838215e+00  1.66997516e+00
  8.70490491e-01 -1.87010527e+00 -6.67819902e-02  5.29245079e-01
 -1.07179570e+00  2.06031382e-01 -1.91458911e-01 -3.80754441e-01
  5.50039887e-01 -5.88886797e-01 -1.19147933e+00  4.65587348e-01
  2.22183332e-01  2.11360

In [27]:
print(word2vec_model['self-indulgent'])

KeyError: "Key 'self-indulgent' not present"

self-indulgent는 학습하지 않았기에 에러가 출력됩니다.

### **2) 사전 훈련된 Word2Vec을 초기 임베딩으로 사용하기**

이제 이 임베딩 벡터들을 IMDB 리뷰 데이터의 단어들에 맵핑해보겠습니다.

In [28]:
import torch
import torch.nn as nn
from torchtext.vocab import Vectors

In [30]:
vectors = Vectors(name="../model/eng_w2v") # 사전 훈련된 모델을 vectors에 저장

100%|██████████| 21613/21613 [00:01<00:00, 11795.76it/s]


Field 객체의 build_vocab을 통해 훈련 데이터의 단어 조합(vocabulary)를 만드는 것과 동시에 임베딩 벡터값들을 초기화할 수 있습니다.

In [31]:
# word2vec 모델을 임베딩 벡터값으로 초기화
TEXT.build_vocab(trainset, vectors=vectors, max_size=10000, min_freq=10)

build_vocab의 인자들을 먼저 보겠습니다.

- max_size: 단어 집합의 크기를 제한
- min_freq: 최소 등장 빈도수를 넘는 단어만 허용
- vectors: 만들어진 단어 집합의 각 단어의 임베딩 벡터값으로 env_w2v에 저장되어져 있던 임베딩 벡터값들로 초기화

단어 집합을 만들었다면, TEXT.vocab.stoi를 통해서 현재 단어 집합의 단어와 맵핑된 고유 정수를 출력할 수 있습니다.

In [35]:
len(TEXT.vocab.stoi), TEXT.vocab.stoi

(10002,
 defaultdict(<bound method Vocab._default_unk_index of <torchtext.legacy.vocab.Vocab object at 0x0000023A788FAB50>>,
             {'<unk>': 0,
              '<pad>': 1,
              'the': 2,
              'a': 3,
              'and': 4,
              'of': 5,
              'to': 6,
              'is': 7,
              'in': 8,
              'i': 9,
              'this': 10,
              'that': 11,
              'it': 12,
              '/><br': 13,
              'was': 14,
              'as': 15,
              'for': 16,
              'with': 17,
              'but': 18,
              'on': 19,
              'movie': 20,
              'his': 21,
              'are': 22,
              'not': 23,
              'film': 24,
              'you': 25,
              'have': 26,
              'he': 27,
              'be': 28,
              'at': 29,
              'one': 30,
              'by': 31,
              'an': 32,
              'they': 33,
              'from': 34,
           

0번부터 10,001번까지 총 10,002개의 단어가 존재합니다. 단, <\unk\>와 <\pad\>는 실제 단어가 아니라 특별 토큰이므로 실제 훈련 데이터의 단어는 10,000개입니다. 

In [36]:
print('임베딩 벡터의 개수와 차원 : {}'.format(TEXT.vocab.vectors.shape))

임베딩 벡터의 개수와 차원 : torch.Size([10002, 100])


임베딩 벡터는 총 10,002개가 존재하며 각각 100차원씩 가지고 있습니다. 

In [37]:
print(TEXT.vocab.vectors[0]) # <unk>의 임베딩 벡터값

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])


In [39]:
print(TEXT.vocab.vectors[1]) # <pad>의 임베딩 벡터값

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])


<\unk>와 <\pad>의 임베딩 벡터값은 0으로 초기화 된 것을 확인할 수 있습니다. 이번엔 this를 확인해보겠습니다.

In [40]:
print(TEXT.vocab.vectors[10]) # this의 임베딩 벡터값

tensor([-5.7152e-02, -1.9514e-01, -7.4318e-01, -9.0819e-01, -3.9966e-01,
         1.7175e+00, -7.5499e-01, -6.7676e-01, -2.4506e-01, -3.8938e-01,
         7.5943e-01,  1.8009e+00, -2.2372e+00, -5.7079e-01, -1.7173e+00,
         1.1195e+00,  2.5929e-01,  9.1094e-01,  7.7065e-01,  3.3547e-01,
         9.8952e-01,  1.4518e+00,  1.4930e+00, -1.6287e-01,  7.2629e-01,
        -8.9633e-01, -1.0216e+00,  3.9209e-01,  1.3144e+00,  7.3766e-01,
         1.3116e+00, -3.6070e-01, -5.3626e-01, -1.4344e+00,  2.7545e-01,
        -1.5321e+00, -1.5151e+00, -6.9013e-02,  3.1930e-01, -1.3167e+00,
        -9.0968e-01, -1.3483e+00, -1.0547e+00, -1.0321e+00,  8.9581e-02,
        -2.0131e+00, -1.3084e+00,  1.6700e+00,  8.7049e-01, -1.8701e+00,
        -6.6782e-02,  5.2925e-01, -1.0718e+00,  2.0603e-01, -1.9146e-01,
        -3.8075e-01,  5.5004e-01, -5.8889e-01, -1.1915e+00,  4.6559e-01,
         2.2218e-01,  2.1136e-03,  1.3678e+00,  1.3999e+00, -2.0341e+00,
         7.8592e-01, -6.9913e-02, -2.0577e-01, -1.7

앞서 저장되어있던 eng_w2v와 같은 값을 가집니다. 

In [41]:
print(TEXT.vocab.vectors[10000]) # 단어 'self-indulgent'의 임베딩 벡터값

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])


기존의 eng_w2v에 존재하지 않기 때문에 임베딩 벡터값도 0으로 초기화되었습니다. 이제 이 임베딩 벡터들을 nn.Embedding()의 초기화 입력으로 사용합시다.

In [42]:
embedding_layer = nn.Embedding.from_pretrained(TEXT.vocab.vectors, freeze=False)

In [43]:
print(embedding_layer(torch.LongTensor([10]))) # 단어 this의 임베딩 벡터값

tensor([[-5.7152e-02, -1.9514e-01, -7.4318e-01, -9.0819e-01, -3.9966e-01,
          1.7175e+00, -7.5499e-01, -6.7676e-01, -2.4506e-01, -3.8938e-01,
          7.5943e-01,  1.8009e+00, -2.2372e+00, -5.7079e-01, -1.7173e+00,
          1.1195e+00,  2.5929e-01,  9.1094e-01,  7.7065e-01,  3.3547e-01,
          9.8952e-01,  1.4518e+00,  1.4930e+00, -1.6287e-01,  7.2629e-01,
         -8.9633e-01, -1.0216e+00,  3.9209e-01,  1.3144e+00,  7.3766e-01,
          1.3116e+00, -3.6070e-01, -5.3626e-01, -1.4344e+00,  2.7545e-01,
         -1.5321e+00, -1.5151e+00, -6.9013e-02,  3.1930e-01, -1.3167e+00,
         -9.0968e-01, -1.3483e+00, -1.0547e+00, -1.0321e+00,  8.9581e-02,
         -2.0131e+00, -1.3084e+00,  1.6700e+00,  8.7049e-01, -1.8701e+00,
         -6.6782e-02,  5.2925e-01, -1.0718e+00,  2.0603e-01, -1.9146e-01,
         -3.8075e-01,  5.5004e-01, -5.8889e-01, -1.1915e+00,  4.6559e-01,
          2.2218e-01,  2.1136e-03,  1.3678e+00,  1.3999e+00, -2.0341e+00,
          7.8592e-01, -6.9913e-02, -2.

## **3. 토치텍스트에서 제공하는 사전 훈련된 워드 임베딩**

토치텍스트는 영어 단어들의 사전 훈련된 임베딩 벡터를 제공하고 있습니다. 그 중, 우리는 glove.6B.300d를 사용하겠습니다. 

IMDB 리뷰 데이터에 존재하는 단어들을 토치텍스트가 제공하는 사전 훈련되 임베딩 벡터들의 값으로 초기화해봅시다.

In [45]:
from torchtext.vocab import GloVe

Field 객체의 build_vocab을 통해 토치텍스트가 제공하는 사전 훈련된 임베딩 벡터를 다운받을 수 있습니다.

In [46]:
TEXT.build_vocab(trainset, vectors=GloVe(name='6B', dim=300), max_size=10000, min_freq=10)
LABEL.build_vocab(trainset)

.vector_cache\glove.6B.zip: 862MB [10:12, 1.41MB/s]                               
100%|█████████▉| 399999/400000 [00:48<00:00, 8241.56it/s]


위 코드는 GloVe의 300차원의 임베딩 벡터들을 다운로드받아 임베딩 벡터 초기화에 사용합니다. 다른 인자들은 IMDB 리뷰 데이터에 있는 단어들을 몇 개만 남길 것인지 정합니다. max_size는 단어 집합의 크기를 제한하고, min_freq=10은 등장 빈도수가 10번 이삭인 단어만 허용합니다. 앞에서 한 것과 동일하게 TEXT.vocab.stoi를 통해서 현재 단어 집합의 단어와 맵핑된 고유한 정수를 출력해보겠습니다.

In [48]:
TEXT.vocab.stoi

defaultdict(<bound method Vocab._default_unk_index of <torchtext.legacy.vocab.Vocab object at 0x0000023A8653ED30>>,
            {'<unk>': 0,
             '<pad>': 1,
             'the': 2,
             'a': 3,
             'and': 4,
             'of': 5,
             'to': 6,
             'is': 7,
             'in': 8,
             'i': 9,
             'this': 10,
             'that': 11,
             'it': 12,
             '/><br': 13,
             'was': 14,
             'as': 15,
             'for': 16,
             'with': 17,
             'but': 18,
             'on': 19,
             'movie': 20,
             'his': 21,
             'are': 22,
             'not': 23,
             'film': 24,
             'you': 25,
             'have': 26,
             'he': 27,
             'be': 28,
             'at': 29,
             'one': 30,
             'by': 31,
             'an': 32,
             'they': 33,
             'from': 34,
             'all': 35,
             'who': 36,
       

In [50]:
print("임베딩 벡터의 개수와 차원 : {}".format(TEXT.vocab.vectors.shape))

임베딩 벡터의 개수와 차원 : torch.Size([10002, 300])


현재 임베딩 벡터는 총 10,002개가 존재하며 각 단어는 300차원을 가집니다.

이제 마찬가지로 nn.Embedding()의 초기화 입력으로 사용하겠습니다.

In [51]:
embedding_layer = nn.Embedding.from_pretrained(TEXT.vocab.vectors, freeze=False)